In [3]:
import pandas as pd
import numpy as np

from class_pig import pigClass  

import glob
import pickle

"""
CUIDADO!!
"""
import warnings
warnings.filterwarnings("ignore")

In [4]:
from fitterClass import funcFitter
from utils import find_best_b, plot_exponentials

In [5]:
def sigmoid(x, a, b, c, d):
    return a + b/(1 + np.exp(-(x-c)/d))

def exponential(x, a, b, c, d):
    return a - b*np.exp(-(x-c)/d)

def exponential_cicle_fixed_b(x, a, c):
    return -a*np.exp(-best_b*x)+c

estimators = ["lm", "trf", "dogbox"]
funcs = [sigmoid, exponential]

In [7]:
dataframes_lst = []

for file_name in glob.glob('porquinhos/*'):
    print(file_name)
    with open(file_name, "rb") as file:
        
        porquinho = pickle.load(file)
        for manobra in porquinho: #Para cada manobra 

            if manobra == "D": # Posso fazer 3,4,5 passos
                n_steps = 5
            elif manobra == "C": # Posso fazer 3,4 passos
                n_steps = 4
            elif manobra == "B": # Posso fazer 3 passos
                n_steps = 3
                
            # Formato os dados de entrada
            format_data = []
            for pi, pe, wi, we in zip(porquinho[manobra]["p_i"], porquinho[manobra]["p_e"],
                                      porquinho[manobra]["w_i"], porquinho[manobra]["w_e"]):

                format_data.extend([pi,wi,pe,we])

            format_data = np.array(format_data).reshape(-1,2)
            ##########################################################
            
            for n_point in range(3,n_steps+1):
                fitter = funcFitter(porquinho.name, manobra, format_data[0::2, :], n_point = n_point, estimators = estimators)

                print(f"Porquinho: {porquinho.name}\nManobra: {manobra}\n Qtd Passo: {n_steps}\n")
                dataframes_lst.append(fitter.fit(funcs=funcs, interpolate=True, n_interp_point=20, interp_method="linear"))

porquinhos/porquinho_mra26.pickle
Porquinho: mra26
Manobra: B
 Qtd Passo: 3

Porquinho: mra26
Manobra: D
 Qtd Passo: 5

Porquinho: mra26
Manobra: D
 Qtd Passo: 5

Porquinho: mra26
Manobra: D
 Qtd Passo: 5

Porquinho: mra26
Manobra: C
 Qtd Passo: 4

Porquinho: mra26
Manobra: C
 Qtd Passo: 4

porquinhos/porquinho_mra29.pickle
Porquinho: mra29
Manobra: D
 Qtd Passo: 5

Porquinho: mra29
Manobra: D
 Qtd Passo: 5

Porquinho: mra29
Manobra: D
 Qtd Passo: 5

Porquinho: mra29
Manobra: B
 Qtd Passo: 3

Porquinho: mra29
Manobra: C
 Qtd Passo: 4

Porquinho: mra29
Manobra: C
 Qtd Passo: 4

porquinhos/porquinho_mra28.pickle
Porquinho: mra28
Manobra: C
 Qtd Passo: 4

Porquinho: mra28
Manobra: C
 Qtd Passo: 4

Porquinho: mra28
Manobra: B
 Qtd Passo: 3

Porquinho: mra28
Manobra: D
 Qtd Passo: 5

Porquinho: mra28
Manobra: D
 Qtd Passo: 5

Porquinho: mra28
Manobra: D
 Qtd Passo: 5



In [8]:
pd.concat(dataframes_lst)\
  .rename(columns = {"function":"prof_example"})\
  .groupby(["subject", "manobra", "n_point"])\
  [["prof_example"]].count()

prof_example
subject manobra n_point              
mra26   B       3                   8
        C       3                   7
                4                   7
        D       3                   9
                4                  10
                5                  12
mra28   B       3                  11
        C       3                   9
                4                  11
        D       3                   9
                4                   9
                5                  10
mra29   B       3                  11
        C       3                  10
                4                  11
        D       3                  10
                4                   9
                5                  11